# Workstreams

1. Web App Development

2. Generate additional test images (via DALL E 3 or StyleGAN)

## Web Application

In [ ]:
from flask import Flask, request, jsonify
import requests

In [ ]:
app = Flask(__name__)

In [ ]:
@app.route('/')
def index():
    return 'Welcome to the Luminare API'

@app.route('/predict', methods=['POST'])
def predict():
    data = request.json 
    model_input = data['input']
    
    # Make a request to AWS Lambda API
    api_url = 'AWS_API_GATEWAY_URL'
    response = requests.post(api_url, json={'input': model_input})
    
    if response.status_code == 200:
        result = response.json()
        return jsonify({'result': result})
    
    else:
        return jsonify({'error': 'Failed to get prediction'})

if __name__ == '__main__':
    app.run(debug=True)

## Model Development

In [ ]:
import openai
import requests
import numpy as np
import os
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

In [ ]:
openai.api_key = ''

# Generate synthetic images using DALL-E 3
def generate_images(prompt, num_images=100):
    response = openai.Image.create(
        model="image-alpha-001",
        prompt=prompt,
        n=num_images,
        size="256x256",
    )

    image_urls = [item['url'] for item in response['data']]
    return image_urls

# Download and save images
def download_images(image_urls, folder, label):
    os.makedirs(folder, exist_ok=True)
    for i, url in enumerate(image_urls):
        img_data = requests.get(url).content
        with open(f'{folder}/{label}_{i}.png', 'wb') as handler:
            handler.write(img_data)

# Generate synthetic images labeled as "fake"
fake_prompt = "A collection of fake images"
fake_images = generate_images(fake_prompt)
download_images(fake_images, 'dataset', 'fake')

In [ ]:
# NEXT STEP: Download real images
# real_images = download_real_images()
# download_images(real_images, 'dataset', 'real')

# Create labels
fake_labels = ['fake'] * len(fake_images)
# real_labels = ['real'] * len(real_images)

# Combine synthetic and real images and labels
# all_images = fake_images + real_images
all_labels = fake_labels  # + real_labels

# Split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    np.array(fake_images), np.array(fake_labels), test_size=0.2, random_state=42
)

In [ ]:
# Train a logistic regression model for evaluation
# Can replace this with Saket or Cheick's pre-existing models
def preprocess_images(image_paths):
    images = [Image.open(path).convert("RGB").resize((64, 64)) for path in image_paths]
    return np.array([np.array(img) for img in images])

X_train_processed = preprocess_images(X_train)
X_test_processed = preprocess_images(X_test)

# Flatten the images for logistic regression
X_train_flat = X_train_processed.reshape(X_train_processed.shape[0], -1)
X_test_flat = X_test_processed.reshape(X_test_processed.shape[0], -1)

# Convert labels to numerical values
le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)
y_test_encoded = le.transform(y_test)

# Train a logistic regression model
model = LogisticRegression()
model.fit(X_train_flat, y_train_encoded)

In [ ]:
# Evaluate the model
y_pred = model.predict(X_test_flat)
accuracy = accuracy_score(y_test_encoded, y_pred)

print(f"Accuracy: {accuracy}")
print("Classification Report:\n", classification_report(y_test_encoded, y_pred))